# 報告 2022-10-19

In [15]:
%%sh
cat ~/ABCT/comp-proto/Annotation-complete-IDed/BCCWJ-ABC*.psd | munge-trees -w > /tmp/comp-yori.psd
cat ~/ABCT/comp-proto/Annotation-complete-IDed/bccwj_kurabe_*.psd | munge-trees -w > /tmp/comp-kurabe.psd

In [16]:
# 「より」の文数
COUNT_YORI, *_ = ! cat /tmp/comp-yori.psd | wc -l
COUNT_YORI = int(COUNT_YORI)

# 「より」のうち，単文の数
# NOTE: CPに関しては，全て単文であることを目視済み。
# NOTE: tregex options: -s: one-liner, -w: whole tree
COUNT_YORI_SIMPLE, *_ = ! tregex -s -w '/^(VPm|VPsub|Sm|Ssub|CP)/ == /root/' /tmp/comp-yori.psd 2> /dev/null | sort | uniq | wc -l 
COUNT_YORI_SIMPLE = int(COUNT_YORI_SIMPLE)

# 「より」のうち，連用節の数
COUNT_YORI_ADVERBIAL, *_ = ! tregex -s -w '/^(VPa|Sa)/ == /root/' /tmp/comp-yori.psd 2> /dev/null | sort | uniq | wc -l 
COUNT_YORI_ADVERBIAL = int(COUNT_YORI_ADVERBIAL)

# 「より」のうち，連体節の数
COUNT_YORI_ADNOMINAL, *_ =  ! tregex -s -w '/^(VPrel|Srel|N)/ == /root/' /tmp/comp-yori.psd 2> /dev/null | sort | uniq | wc -l 
COUNT_YORI_ADNOMINAL = int(COUNT_YORI_ADNOMINAL)

# そもそも比較構文でない物の数
COUNT_YORI_NA, *_ = ! cat /tmp/comp-yori.psd | sed -e '/#comp/d' | wc -l
COUNT_YORI_NA = int(COUNT_YORI_NA)


In [17]:
# 「比べて」の文数
COUNT_KURABE, *_ = ! cat /tmp/comp-kurabe.psd | wc -l
COUNT_KURABE = int(COUNT_KURABE)

# 「比べて」のうち，単文の数
# NOTE: CPに関しては，全て単文であることを目視済み。
# NOTE: tregex options: -s: one-liner, -w: whole tree
COUNT_KURABE_SIMPLE, *_ = ! tregex -s -w '/^(VPm|VPsub|Sm|Ssub|CP)/ == /root/' /tmp/comp-kurabe.psd 2> /dev/null | sort | uniq | wc -l 
COUNT_KURABE_SIMPLE = int(COUNT_KURABE_SIMPLE)

# 「比べて」のうち，連用節の数
COUNT_KURABE_ADVERBIAL, *_ = ! tregex -s -w '/^(VPa|Sa)/ == /root/' /tmp/comp-kurabe.psd 2> /dev/null | sort | uniq | wc -l 
COUNT_KURABE_ADVERBIAL = int(COUNT_KURABE_ADVERBIAL)

# 「比べて」のうち，連体節の数
COUNT_KURABE_ADNOMINAL, *_ =  ! tregex -s -w '/^(VPrel|Srel|N)/ == /root/' /tmp/comp-kurabe.psd 2> /dev/null | sort | uniq | wc -l 
COUNT_KURABE_ADNOMINAL = int(COUNT_KURABE_ADNOMINAL)

# そもそも比較構文でない物の数
COUNT_KURABE_NA, *_ = ! cat /tmp/comp-kurabe.psd | sed -e '/#comp/d' | wc -l
COUNT_KURABE_NA = int(COUNT_KURABE_NA)

In [18]:
# 集計
import pandas as pd

STAT = pd.DataFrame(
    {
        "全文数": [COUNT_YORI, COUNT_KURABE],
        "連用節数": [COUNT_YORI_ADVERBIAL, COUNT_KURABE_ADVERBIAL],
        "連体節数": [COUNT_YORI_ADNOMINAL, COUNT_KURABE_ADNOMINAL],
        "その他比較構文数": [
            COUNT_YORI - COUNT_YORI_ADVERBIAL - COUNT_YORI_ADNOMINAL - COUNT_YORI_NA,
            COUNT_KURABE - COUNT_KURABE_ADVERBIAL - COUNT_KURABE_ADNOMINAL - COUNT_KURABE_NA,
        ],
        "比較構文でない数": [COUNT_YORI_NA, COUNT_KURABE_NA],
    },
    index = ["より", "比べて"]
)

In [19]:
STAT

,全文数,連用節数,連体節数,その他比較構文数,比較構文でない数
より,2700,288,449,1108,855
比べて,1042,123,89,552,278


In [20]:
# 合計
STAT.sum()

全文数         3742
連用節数         411
連体節数         538
その他比較構文数    1660
比較構文でない数    1133
dtype: int64

## 学習データ／評価データ
9:1になるように，事前に分割した。

In [21]:
import datasets

# NOTE: private repoなので，事前にログインが必要。
ds = datasets.load_dataset(
    "abctreebank/comparative-NER-BCCWJ",
    use_auth_token = True,
)

ds

Using custom data configuration abctreebank--comparative-NER-BCCWJ-f36bf31580d9adae
Reusing dataset parquet (/home/owner/.cache/huggingface/datasets/abctreebank___parquet/abctreebank--comparative-NER-BCCWJ-f36bf31580d9adae/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 2/2 [00:00<00:00, 321.17it/s]


DatasetDict({
    test: Dataset({
        features: ['ID', 'tokens', 'comp'],
        num_rows: 374
    })
    train: Dataset({
        features: ['ID', 'tokens', 'comp'],
        num_rows: 3368
    })
})

* 学習データ数： 3,368文
* テストデータ数： 374文

### 学習データの例

#### 単文
```
5_BCCWJ-ABC-aa-simple
妻 が 仕事 に 精出す 一方 、 [[赤沼 は]cont [それ より]prej [もっと]diff [忙しい]deg 。]root

5_BCCWJ-ABC-aa-simple_predicted
[CLS] 妻 が 仕事 に 精 ##出す 一方 、 [赤 ##沼 は]cont [それ より]prej [もっと]diff [忙 ##しい]deg 。 [SEP]


95_bccwj_kurabe_text-aj-simplified
[[旧法 が 成立 し た 当時 に 比べ て]prej 、 私 たち の 食生活 は [格段 に]diff [豊か]deg に なっ た 。]root

95_bccwj_kurabe_text-aj-simplified_predicted
[CLS] [旧 ##法 が 成立 し た 当時 に 比べ て]prej 、 私 たち の 食 ##生活 は [格段 に]diff [豊か]deg に なっ た 。 [SEP]

35_bccwj_kurabe_text-ah-simplified
[[アカ ナマコ の 成長 は]cont [アオナマコ に]prej [比べ]prej [若干]diff [劣っ]deg て い た 。]root

35_bccwj_kurabe_text-ah-simplified_predicted
[CLS] [アカ ナ ##マコ の]cont 成長 は [アオ ##ナ ##マコ に 比べ]prej [若干]diff [劣っ]deg て い た 。 [SEP]
```

#### 連用節
```
23_bccwj_kurabe_text-af-simplified
ところが 、 一 九 八 五 年 九月 の プラザ 合意 以降 、 [[円 が]cont [ドル に 比べ て]prej [百 ％ 以上]diff [はね上がり]deg]root 、 突然 日本 は アメリカ より はるか に 高 コスト の 国 に なり まし た 。

23_bccwj_kurabe_text-af-simplified_predicted
[CLS] ところが 、 一 九 八 五 年 九 ##月 の プラザ 合意 以降 、 [円 が]cont [ドル に 比べ て]prej [百 % 以上]diff [はね ##上がり]deg 、 突然 [日本 は]cont [アメリカ より]prej [はるか に]diff [高 コスト]deg の 国 に なり まし た 。 [SEP]
```

#### 連体節
```
21_BCCWJ-ABC-as-simple
三 十 歳 の サラリーマン が [自分 より]prej [七]diff [、]diff [[[八 歳]diff [年下]deg]cont]root の 「 新入 社員 の 気持ち が わから ない 」 と 言っ て いる 。

21_BCCWJ-ABC-as-simple_predicted
[CLS] 三 十 歳 の サラリーマン [が]cont [自分 より]prej [七 、 八 歳]diff [年下]deg の 「 新入 社員 の 気持ち が わから ない 」 と 言っ て いる 。 [SEP]
```
（ `[が]cont` が変）

```
99_BCCWJ-ABC-au-simple
何 畳 ある か わから ない くらい 、 [[[教室 より も]prej [広い]deg]cont]root 部屋 。

99_BCCWJ-ABC-au-simple_predicted
[CLS] 何 畳 ある か わから ない くらい 、 [教室 より も]prej [広い]deg 部屋 。 [SEP]
```

## モデルの設定
* 使用した事前学習モデル： https://huggingface.co/cl-tohoku/bert-base-japanese-whole-word-masking
* このモデルの上で，NER（固有表現認識）の一種として，与えられた文のどのspanが，比較構文の要素に相当するのかについてのモデルを構築。
* 比較構文の要素：
    * prej(acent)：「より」句
    * cont(rast)：比較対象
    * diff(erence)：差の表現
    * deg(ree)：程度表現
    * root：比較構文の最大スコープ（NERモデルにおいては取り除いた）
* 例： [ [太郎が]cont [花子よりも]prej [3cm]diff [高い]deg ]root ことは意外だった。

In [22]:
# 学習パラメータ

training_args = dict(
    # output_dir = str(output_path),

    # エポック数
    num_train_epochs = 27,

    # バッチのサイズ
    per_device_train_batch_size = 64,
    per_device_eval_batch_size = 128,

    # 学習率
    learning_rate = 5e-5,
    
    warmup_steps = 200,
    weight_decay = 0,
    # save_strategy = IntervalStrategy.STEPS,
    save_steps = 1000,
    do_eval = True,
    # evaluation_strategy = IntervalStrategy.STEPS,
    eval_steps = 109,
    include_inputs_for_metrics = True,

    # 乱数シード
    seed = 2630987289,

    # logging_dir = str(output_path / "logs"),
    logging_steps= 10,
)

## 学習結果

In [23]:
result = {"score_spanwise_details": {"cont": {"CORRECT": 107, "WRONG_SPAN": 73, "SPURIOUS": 67, "MISSING": 53, "WRONG_LABEL": 1, "WRONG_LABEL_SPAN": 9}, "prej": {"CORRECT": 229, "WRONG_LABEL_SPAN": 6, "WRONG_SPAN": 33, "MISSING": 5, "SPURIOUS": 32}, "diff": {"CORRECT": 63, "MISSING": 10, "SPURIOUS": 14, "WRONG_SPAN": 10, "WRONG_LABEL_SPAN": 1}, "deg": {"CORRECT": 218, "WRONG_SPAN": 15, "WRONG_LABEL": 3, "SPURIOUS": 60, "MISSING": 34, "WRONG_LABEL_SPAN": 4}}, "score_spanwise": {"cont": {"possible_entries": 243, "actual_entries": 257, "precision_strict": 0.4163424124513619, "recall_strict": 0.4403292181069959, "F1_strict": 0.574, "precision_partial": 0.5583657587548638, "recall_partial": 0.5905349794238683}, "prej": {"possible_entries": 273, "actual_entries": 300, "precision_strict": 0.7633333333333333, "recall_strict": 0.8388278388278388, "F1_strict": 0.8568935427574172, "precision_partial": 0.8183333333333334, "recall_partial": 0.8992673992673993}, "diff": {"possible_entries": 84, "actual_entries": 88, "precision_strict": 0.7159090909090909, "recall_strict": 0.75, "F1_strict": 0.7906976744186046, "precision_partial": 0.7727272727272727, "recall_partial": 0.8095238095238095}, "deg": {"possible_entries": 274, "actual_entries": 300, "precision_strict": 0.7266666666666667, "recall_strict": 0.7956204379562044, "F1_strict": 0.7857142857142858, "precision_partial": 0.7516666666666667, "recall_partial": 0.822992700729927}}, "score_spanwise_F1_strict": 0.7518263757225769, "score_tokenwise": {"IGNORE": {"precision": 0.0, "recall": 0.0, "f1-score": 0.0, "support": 0}, "O": {"precision": 0.9349922839506173, "recall": 0.9109190001879346, "f1-score": 0.9227986673012851, "support": 5321}, "B-deg": {"precision": 0.761437908496732, "recall": 0.8411552346570397, "f1-score": 0.7993138936535162, "support": 277}, "B-prej": {"precision": 0.8160535117056856, "recall": 0.8591549295774648, "f1-score": 0.83704974271012, "support": 284}, "B-cont": {"precision": 0.6639004149377593, "recall": 0.6986899563318777, "f1-score": 0.6808510638297872, "support": 229}, "B-diff": {"precision": 0.7419354838709677, "recall": 0.7752808988764045, "f1-score": 0.7582417582417582, "support": 89}, "I-deg": {"precision": 0.6923076923076923, "recall": 0.7397260273972602, "f1-score": 0.7152317880794701, "support": 73}, "I-prej": {"precision": 0.8496, "recall": 0.959349593495935, "f1-score": 0.9011455239711498, "support": 1107}, "I-cont": {"precision": 0.7381703470031545, "recall": 0.6733812949640288, "f1-score": 0.7042889390519188, "support": 695}, "I-diff": {"precision": 0.7654320987654321, "recall": 0.6813186813186813, "f1-score": 0.7209302325581395, "support": 91}, "micro avg": {"precision": 0.881582169973059, "recall": 0.881582169973059, "f1-score": 0.881582169973059, "support": 8166}, "macro avg": {"precision": 0.6963829741038041, "recall": 0.7138975616806627, "f1-score": 0.7039851609397145, "support": 8166}, "weighted avg": {"precision": 0.8828759818341678, "recall": 0.881582169973059, "f1-score": 0.8814116671139006, "support": 8166}}}

In [24]:
# データカウント
df_res_count = pd.DataFrame.from_dict(
    result["score_spanwise_details"],
    orient = "index",
)

df_res_count

,CORRECT,WRONG_SPAN,SPURIOUS,MISSING,WRONG_LABEL,WRONG_LABEL_SPAN
cont,107,73,67,53,1.0,9
prej,229,33,32,5,NaN,6
diff,63,10,14,10,NaN,1
deg,218,15,60,34,3.0,4


* CORRECT: ぴったり
* WRONG_SPAN: spanにずれがある。
* SPURIOUS: 正解データにないspanを予測してしまっている。
* MISSING: 正解データにあるspanを予測できていない。
* WRONG_LABEL_SPAN: spanにずれがあり，かつ，ラベルも間違っている。
* WRONG_LABEL: spanにずれはないが，ラベルが間違っている。

In [25]:
# 統計
df_res_stat = pd.DataFrame.from_dict(
    result["score_spanwise"],
    orient = "index",
)

df_res_stat

,possible_entries,actual_entries,precision_strict,recall_strict,F1_strict,precision_partial,recall_partial
cont,243,257,0.416342,0.440329,0.574000,0.558366,0.590535
prej,273,300,0.763333,0.838828,0.856894,0.818333,0.899267
diff,84,88,0.715909,0.750000,0.790698,0.772727,0.809524
deg,274,300,0.726667,0.795620,0.785714,0.751667,0.822993


### 凡例

* `possible_entries`: 予測されたspanの数
    ```
    ct["CORRECT"] + ct["WRONG_SPAN"] + ct["WRONG_LABEL"] + ct["WRONG_LABEL_SPAN"] + ct["MISSING"]
    ```
* `actual_entires`: テストデータにあるspanの数
    ```
    res["possible_entries"] - ct["MISSING"] + ct["SPURIOUS"]
    ```
* `precision_strict`: 予測のうち，当たっているものの数
    ```
    ct["CORRECT"] / res["actual_entries"]
    ```
* `recall_strict`: テストデータにあるもののうち，予測されたspanの数
    ```
    ct["CORRECT"] / res["possible_entries"]
    ```
* `precision_partial`：strictよりも緩い。 WRONG_SPANを50%カウントに入れている。
    ```
    (ct["CORRECT"] + 0.5 * ct["WRONG_SPAN"]) / res["actual_entries"]
    ```
* `recall_partial`
    ```
    (ct["CORRECT"] + 0.5 * ct["WRONG_SPAN"]) / res["possible_entries"]
    ```
* F1はprecisionとrecallの調和平均

In [26]:
# F1_strictの単純平均。ラベルのカウントで重みづけることはしていない。
result["score_spanwise_F1_strict"]

0.7518263757225769

# 学習結果：連体節なしバージョン

In [27]:
# まず，連体節に該当する文のIDを抜き出す。

TREES_YORI_ADNOMINAL =  ! tregex -s -w '/^(VPrel|Srel|N)/ == /root/' /tmp/comp-yori.psd 2> /dev/null | sort | uniq
TREES_KURABE_ADNOMINAL =  ! tregex -s -w '/^(VPrel|Srel|N)/ == /root/' /tmp/comp-kurabe.psd 2> /dev/null | sort | uniq

TREES_ADNOMINAL = TREES_YORI_ADNOMINAL + TREES_KURABE_ADNOMINAL

import re
_RE_TREE_ID = re.compile(r"\(ID (?P<ID>[^)]+)\)")
IDs_ADNOMINAL = [
    # tree.group("ID")
    tree.group("ID")
    for tree in 
    filter(None, map(_RE_TREE_ID.search, TREES_ADNOMINAL))
]

In [30]:
from typing import Sequence

import numpy as np
import torch
import torch.utils.data

import datasets
from transformers import BertConfig, BertForTokenClassification, BertJapaneseTokenizer, EvalPrediction, IntervalStrategy, TokenClassificationPipeline, Trainer, TrainingArguments
import evaluate

from abct_comp_ner_utils.train import _add_vectors, ID2LABEL_DETAILED, LABEL2ID, _get_tokenizer, _get_evaluator

model = BertForTokenClassification.from_pretrained("../results_2022-10-19")

assert(isinstance(model, BertForTokenClassification))

dataset = datasets.load_dataset(
    "abctreebank/comparative-NER-BCCWJ",
    use_auth_token = True,
    split = "test",
)
assert(isinstance(dataset, datasets.Dataset))

# 連体節を排除
dataset = dataset.filter(
    lambda x: x["ID"] not in IDs_ADNOMINAL
)

dataset = dataset.map(
    _add_vectors,
)

model.eval()

def _make_spans(
    input: Sequence | np.ndarray,
    pred: Sequence | np.ndarray
):
    result = {
        "start": [],
        "end": [],
        "label": [],
    }

    current_label = ID2LABEL_DETAILED[0][0]
    current_span_start: int = 0

    for loc, (input_id, label_id) in enumerate(zip(input, pred)):
        label = ID2LABEL_DETAILED[label_id][0]

        if input_id == 0:
            # reached padding
            break
        elif current_label != label:
            # label changed
            # conclude the old label
            if current_label not in ("IGNORE", "O"):
                result["start"].append(current_span_start)
                result["end"].append(loc)
                result["label"].append(current_label)
            else:
                pass

            # switch to new label
            current_label = label
            current_span_start = loc

    return result

def _decode(tokens):
    tokens_decoded = _get_tokenizer().batch_decode(
        [t for t in tokens if t != 0],
        skip_special_tokens = True,
    )

    return [t.replace(" ", "") for t in tokens_decoded]

def _predict(
    examples: datasets.arrow_dataset.Example | datasets.arrow_dataset.Batch
):
    examples["tokens_re"] = [
        _decode(entry) for entry in examples["input_ids"]
    ]

    predictions_raw = model.forward(
        input_ids = torch.tensor(examples["input_ids"]),
        attention_mask = torch.tensor(examples["attention_mask"]),
        token_type_ids = torch.tensor(examples["token_type_ids"]),
    ).logits
    match predictions_raw:
        case torch.Tensor():
            predictions: np.ndarray = predictions_raw.argmax(dim = 2).numpy()
        case np.ndarray():
            predictions: np.ndarray = predictions_raw.argmax(axis = 2)
        case _:
            raise TypeError
    examples["prediction"] = predictions


    examples["comp_predicted"] = [
        _make_spans(i, p)
        for i, p in zip(examples["input_ids"], predictions)
    ]
    
    return examples

dataset = dataset.map(
    _predict,
    batched = True,
    batch_size = 128,
)

_eval: evaluate.Metric = _get_evaluator(
    "../comparative-NER-metrics"
)

res_no_adnom = _eval._compute(
    predictions = dataset["prediction"],
    references = dataset["label_ids"],
    input_ids = dataset["input_ids"],
    special_ids = _get_tokenizer().all_special_ids,
    label2id = LABEL2ID,
    id2label_detailed = ID2LABEL_DETAILED,
)

Using custom data configuration abctreebank--comparative-NER-BCCWJ-f36bf31580d9adae
Reusing dataset parquet (/home/owner/.cache/huggingface/datasets/abctreebank___parquet/abctreebank--comparative-NER-BCCWJ-f36bf31580d9adae/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 3/3 [00:18<00:00,  6.29s/ba]


In [31]:
# データカウント
df_res_count_no_adnom = pd.DataFrame.from_dict(
    res_no_adnom["score_spanwise_details"],
    orient = "index",
)

df_res_count_no_adnom

,CORRECT,WRONG_SPAN,SPURIOUS,MISSING,WRONG_LABEL_SPAN,WRONG_LABEL
cont,87,64,52,35,5,NaN
prej,188,25,31,3,6,NaN
diff,47,7,12,9,1,NaN
deg,180,11,54,28,4,2.0


In [32]:
df_res_stat_no_adnom = pd.DataFrame.from_dict(
    res_no_adnom["score_spanwise"],
    orient = "index",
)

df_res_stat_no_adnom

,possible_entries,actual_entries,precision_strict,recall_strict,F1_strict,precision_partial,recall_partial
cont,191,208,0.418269,0.455497,0.596491,0.572115,0.623037
prej,222,250,0.752000,0.846847,0.849576,0.802000,0.903153
diff,64,67,0.701493,0.734375,0.770992,0.753731,0.789062
deg,225,251,0.717131,0.800000,0.779412,0.739044,0.824444


In [33]:
# F1_strictの単純平均。ラベルのカウントで重みづけることはしていない。
res_no_adnom["score_spanwise_F1_strict"]

0.749117907593678